# Python for Finance: Beta Weighting your Portfolio
 we begin by using yfinance to import financial stock data.

In [1]:
## This is required for pandas_datareader on google colab - then you need to restart runtime
#!pip install --upgrade pandas_datareader

In [2]:
import datetime as dt
import pandas as pd
import numpy as np
from scipy import stats
from pandas_datareader import data as pdr
import yfinance as yf
from pandas_datareader import data as pdr
import plotly.offline as pyo
import plotly.graph_objects as go
from plotly.subplots import make_subplots
yf.pdr_override()

#### Step 1: Specify date range for analysis
Here we begin by creating start and end dates using pythons datetime module.

In [3]:
start = dt.datetime(2010, 1, 1)
end = dt.datetime.now()
start, end

(datetime.datetime(2010, 1, 1, 0, 0),
 datetime.datetime(2024, 1, 27, 13, 3, 12, 410882))

#### Step 2: Select the stocks/tickers you would like to analyse
For Australian stocks, yahoo tickers require '.AX' to be specified at the end of the ticker symbol.

For other tickers, use the search bar in yahoo finance to work out other ticker structures.
https://au.finance.yahoo.com/

In [4]:
stockList = ['CBA', 'NAB', 'WBC', 'ANZ']
stocks = ['^AXJO'] + [i + '.AX' for i in stockList]
stocks

['^AXJO', 'CBA.AX', 'NAB.AX', 'WBC.AX', 'ANZ.AX']

#### Step 3 call the Pandas_Datareader DataReader module:
Two ways of doing this:
1. pdr.DataReader(stocks, 'yahoo', start, end)
2. pdr.get_data_yahoo(stocks, start, end)

In [5]:
df = yf.download(stocks, start, end)
log_returns = np.log(df.Close / df.Close.shift(1)).dropna()
log_returns.head()

[*********************100%%**********************]  5 of 5 completed


Ticker,ANZ.AX,CBA.AX,NAB.AX,WBC.AX,^AXJO
Date,,,,,
2010-01-05,0.004794,0.015013,0.012693,0.008266,0.009795
2010-01-06,-0.014893,0.005014,-0.019651,-0.004715,-0.000589
2010-01-07,-0.024119,-0.009693,-0.008118,-0.013482,-0.004480
2010-01-08,0.005860,0.012903,-0.003340,0.003984,0.002589
2010-01-11,0.006272,0.007274,0.012927,0.001192,0.007827


#### Step 4a: Directly calculate beta:

$\frac{covariance(Market, Stock)}{variance(Market)}$

In [6]:
def calc_beta(df):
    np_array = df.values
    # Market index is the first column 0
    m = np_array[:,0]
    beta = []
    for ind, col in enumerate(df):
        if ind > 0:
            # stock returns are indexed by ind
            s = np_array[:,ind]
            # Calculate covariance matrix between stock and market
            covariance = np.cov(s,m)
            beta.append( covariance[0,1]/covariance[1,1] )
    return pd.Series(beta, df.columns[1:], name='Beta')

In [7]:
calc_beta(log_returns)

Ticker
CBA.AX    0.691719
NAB.AX    0.809128
WBC.AX    0.845186
^AXJO     0.525495
Name: Beta, dtype: float64

#### Step 4b: Use linear regression to get coefficient of market and stocks returns

In [8]:
def regression_beta(df):
    np_array = df.values
    # Market index is the first column 0
    m = np_array[:,0]
    beta = []
    for ind, col in enumerate(df):
        if ind > 0:
            s = np_array[:,ind] # stock returns are column one from numpy array
            beta.append( stats.linregress(m,s)[0] )
    return pd.Series(beta, df.columns[1:], name='Beta')

In [9]:
regression_beta(log_returns)

Ticker
CBA.AX    0.691719
NAB.AX    0.809128
WBC.AX    0.845186
^AXJO     0.525495
Name: Beta, dtype: float64

#### Step 4c: Use Matrix Algebra to complete linear regression in one line

For linear regression on a model of the form y=Xβ, where X is a matrix with full column rank, the least squares solution,

$\hat{\beta} = arg \min ||X\beta−y||_2 $


$\hat{\beta} = (X^T X)^{−1}X^Ty $

https://stats.stackexchange.com/questions/23128/solving-for-regression-parameters-in-closed-form-vs-gradient-descent/23132#23132

In [10]:
def matrix_beta(df):
    # Market index is the first column 0
    X = df.values[:, [0]]
    # add an additional column for the intercept (initalise as 1's)
    X = np.concatenate([np.ones_like(X), X], axis=1)
    # Apply matrix algebra for linear regression model
    beta = np.linalg.pinv(X.T @ X) @ X.T @ df.values[:, 1:]
    return pd.Series(beta[1], df.columns[1:], name='Beta')

In [11]:
beta = matrix_beta(log_returns)
beta

Ticker
CBA.AX    0.691719
NAB.AX    0.809128
WBC.AX    0.845186
^AXJO     0.525495
Name: Beta, dtype: float64

#### Step 5: Define your Portfolio and make DataFrame

Calculate Beta Weighted Portfolio

In [12]:
units = np.array([100, 250, 300, 400, 200])
ASXprices = df.Close[-1:].values.tolist()[0]
price = np.array([round(price,2) for price in ASXprices[1:]])
value = [unit*pr for unit, pr in zip(units, price)]
weight = [round(val/sum(value),2) for val in value]
beta = round(beta,2)

#### Step 6: What if we have options, let's consider things in terms of Delta

In [13]:
Portfolio = Portfolio.drop(['Weight', 'Weighted Beta'], axis=1)
Portfolio['Delta'] = Portfolio['Units']
Portfolio

NameError: name 'Portfolio' is not defined

#### Add Options to portfolio
This is Only an example

In [ ]:
Options = [{'option':'CBA0Z8', 'underlying':'CBA', 'price':3.950, 'units': 2, 'delta': 0.627, 'direction': 'Short', 'type': 'Call'},
           {'option':'WPLQB9', 'underlying':'WPL', 'price':1.325, 'units': 2, 'delta': -0.425 ,'direction': 'Long', 'type': 'Put'}]

In [ ]:
for index, row in enumerate(Options):
    Portfolio.loc[row['option']] = [row['underlying'], row['direction'], row['type'], Portfolio.loc[row['underlying']+'.AX', 'Price'],
                                    row['price'], row['units'], row['price']*row['units']*100, beta[row['underlying']+'.AX'],
                                    (row['delta']*row['units']* 100 if row['direction'] == 'Long' else -row['delta']*row['units']*100)]
Portfolio

#### Step 7: Weight the Delta's using Beta

In [ ]:
Portfolio['ASX200 Weighted Delta (point)'] = round(Portfolio['Beta'] * (Portfolio['Stock Price']/ASXprices[0]) * Portfolio['Delta'],2)
Portfolio['ASX200 Weighted Delta (1%)'] = round(Portfolio['Beta'] * (Portfolio['Stock Price']) * Portfolio['Delta'] * 0.01,2)
Portfolio

#### Step 8: Total the Delta's to get Portfolio Overview

In [ ]:
Portfolio.loc['Total', ['Value', 'ASX200 Weighted Delta (point)', 'ASX200 Weighted Delta (1%)']] \
= Portfolio[['Value','ASX200 Weighted Delta (point)', 'ASX200 Weighted Delta (1%)']].sum()
Portfolio